In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras import callbacks
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import load_model
import keras
import keras_tuner
import keras.utils as image
from keras import layers
from keras import ops
from keras import callbacks
from keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler
from PIL import Image
import gc
# del model  # Delete the old model reference
gc.collect() 

0

## Importing Data from csv files


In [2]:
images = pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/images.txt", sep=r'\s+', names=['image_id', 'image_name'], engine='python')
train_test_split = pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/train_test_split.txt", sep=r'\s+', names=['image_id', 'is_training_image'], engine='python')
classes =pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/classes.txt", sep=r'\s+', names=['class_id', 'class_name'], engine='python')
image_class_labels =pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/image_class_labels.txt", sep=r'\s+', names=['image_id', 'class_id'], engine='python')

## Preprocessing

In [3]:
# Merge dfs based on column names so we have one df with all the necessary info contained per each row
image_data = pd.merge(images,train_test_split, on='image_id')
image_data = pd.merge(image_data,image_class_labels, on='image_id')
image_data = pd.merge(image_data,classes, on='class_id')

In [4]:
# Split training and testing image data
training_image_data = image_data[image_data['is_training_image']==1]
testing_image_data = image_data[image_data['is_training_image']==0]

# Shuffle training data
training_image_data = training_image_data.sample(frac=1)

# Initiate empty lists for training and testing images
training_images = []
testing_images = []

# Add training and testing images to corresponding lists
for i in (training_image_data['image_name'].values):
    training_images.append(image.load_img('/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/images/{}'.format(i), target_size=(224, 224)))

for i in (testing_image_data['image_name'].values):
    testing_images.append(image.load_img('/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/images/{}'.format(i), target_size=(224, 224)))

# print(len(training_images)+len(testing_images))
# Extract class labels for training and testing images
training_class_label = np.array(training_image_data['class_id'].values)
testing_class_label = np.array(testing_image_data['class_id'].values)

In [5]:
# Convert list of images to NumPy array
training_images = np.array(training_images)
testing_images = np.array(testing_images)

# Apply preprocessing
preprocessed_training_images = preprocess_input(training_images)
preprocessed_testing_images = preprocess_input(testing_images)

# Define validation data split at 30%
split_idx = int(len(preprocessed_training_images) * 0.7)
X_train, X_val = preprocessed_training_images[:split_idx], preprocessed_training_images[split_idx:]
y_train, y_val = training_class_label[:split_idx], training_class_label[split_idx:]

y_train = np.array(y_train).astype(np.int32)
y_val = np.array(y_val).astype(np.int32)

We begin training by keeping the existing layers frozen, allowing only the newly added classification head to learn. This ensures that the output layer is properly trained before fine-tuning the deeper layers. Once the classifier stabilizes, we gradually unfreeze and fine-tune the earlier layers, optimizing them in a controlled manner. This approach prevents instability in feature extraction and allows each layer to adjust effectively, improving overall model performance.

In [9]:
# Load pre-trained Xception model (without top layers)
base_model = keras.applications.Xception(
    weights="imagenet",  
    include_top=False,  
    input_shape=(224, 224, 3)
)  

base_model.trainable = False

# Create the classification head
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)  # Base model stays frozen
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(5e-4))(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(201, activation="softmax")(x)

# Create final model
model = keras.Model(inputs, outputs)

# Explicitly make the last few layers trainable (AFTER model creation)
for layer in model.layers[-3:]:  # Last 3 layers should be trainable
    layer.trainable = True

# Double-check trainable layers
for layer in model.layers:
    print(layer.name, "Trainable:", layer.trainable)

# Define optimizer
optimizer = keras.optimizers.Adam(learning_rate=1e-4)  

# Set early stopping
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=3, restore_best_weights=True)

# Compile model AFTER setting trainable layers
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Train model
history = model.fit(X_train, y_train, epochs=13, validation_data=(X_val, y_val), batch_size=32, callbacks=[earlystopping])

# Save the trained frozen model
model.save("best_base_frozen_model.h5",  overwrite=True)

input_layer_5 Trainable: True
xception Trainable: False
global_average_pooling2d_2 Trainable: True
dense_4 Trainable: True
dropout_2 Trainable: True
dense_5 Trainable: True
Epoch 1/13
132/132 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.0143 - loss: 5.5163 - val_accuracy: 0.0556 - val_loss: 5.2105
Epoch 2/13
132/132 ━━━━━━━━━━━━━━━━━━━━ 169s 1s/step - accuracy: 0.0651 - loss: 5.0739 - val_accuracy: 0.1245 - val_loss: 4.7223
Epoch 3/13
132/132 ━━━━━━━━━━━━━━━━━━━━ 217s 2s/step - accuracy: 0.1535 - loss: 4.5052 - val_accuracy: 0.1962 - val_loss: 4.2439
Epoch 4/13
132/132 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - accuracy: 0.2106 - loss: 4.0170 - val_accuracy: 0.2518 - val_loss: 3.8516
Epoch 5/13
132/132 ━━━━━━━━━━━━━━━━━━━━ 154s 1s/step - accuracy: 0.2416 - loss: 3.6878 - val_accuracy: 0.2874 - val_loss: 3.5662
Epoch 6/13
132/132 ━━━━━━━━━━━━━━━━━━━━ 154s 1s/step - accuracy: 0.2745 - loss: 3.4498 - val_accuracy: 0.3280 - val_loss: 3.3496
Epoch 7/13
132/132 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step 

In [7]:
# Load the best trained frozen model
model = keras.models.load_model("best_base_frozen_model.h5")

# Unfreeze the last 8 layers of the Xception base model
base_model = model.layers[1]  # Xception model is the second layer in the full model

for layer in base_model.layers[-8:]:  # Correct way to unfreeze last 8 layers
    layer.trainable = True

# for layer in model.layers:
#     print(layer.name, "Trainable:", layer.trainable)
    
# Use a smaller learning rate to avoid destroying pretrained weights
optimizer_finetune = keras.optimizers.Adam(learning_rate=1e-5)

# Recompile the model after making changes
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_finetune, metrics=["accuracy"])

# Print trainable layers (all should now be `True` for the last 8 layers of base model)
# for layer in model.layers[1].layers:
#     print(layer.name, "Trainable:", layer.trainable)  # Double-check trainable status

# Set early stopping
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=3, restore_best_weights=True)

# Fine-tune the entire model for 10 more epochs
print("Fine-tuning the model end-to-end...")
history_finetune = model.fit(
    X_train, y_train,
    epochs=10,  
    validation_data=(X_val, y_val),
    batch_size=32,
    callbacks=[earlystopping]
)

# Save the fine-tuned model
model.save("finetuned_model.h5")

Fine-tuning the model end-to-end...
Epoch 1/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 850ms/step - accuracy: 0.3480 - loss: 3.5118

In [ ]:
# Load the best frozen model
model = keras.models.load_model("best_frozen_model.h5")

# Evaluate on the test set
test_loss, test_acc = model.evaluate(preprocessed_testing_images, testing_class_label)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

In [13]:
# Load pre-trained Xception model (without top layers)
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)

# Add new layers on top
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
# dense = keras.layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(l2=1e-4))(avg)  # L2 regularization
output = keras.layers.Dense(201, activation="softmax")(avg)  # 201 classes

# Create model
model = keras.Model(inputs=base_model.input, outputs=output)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Define optimizer
optimizer = keras.optimizers.Adam()  # Use Adam with a fixed learning rate

# Compile model
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Set early stopping
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=3, restore_best_weights=True)

# Train model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32, callbacks=[earlystopping])

# ✅ Save the trained frozen model
model.save("best_frozen_model.h5")

Epoch 1/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.1135 - loss: 4.6320 - val_accuracy: 0.3346 - val_loss: 2.7695
Epoch 2/10
 26/132 ━━━━━━━━━━━━━━━━━━━━ 1:17 733ms/step - accuracy: 0.5093 - loss: 2.0965

KeyboardInterrupt: 

In [13]:
# Load the best frozen model
model = keras.models.load_model("best_base_frozen_model.keras")

# Evaluate on the test set
test_loss, test_acc = model.evaluate(preprocessed_testing_images, testing_class_label)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

ValueError: A total of 80 objects could not be loaded. Example error message for object <Conv2D name=block1_conv1, built=True>:

Layer 'block1_conv1' expected 2 variables, but received 1 variables during loading. Expected: ['kernel', 'kernel']

List of objects that could not be loaded:
[<Conv2D name=block1_conv1, built=True>, <BatchNormalization name=block1_conv1_bn, built=True>, <Conv2D name=block1_conv2, built=True>, <BatchNormalization name=block1_conv2_bn, built=True>, <SeparableConv2D name=block2_sepconv1, built=True>, <BatchNormalization name=block2_sepconv1_bn, built=True>, <SeparableConv2D name=block2_sepconv2, built=True>, <BatchNormalization name=block2_sepconv2_bn, built=True>, <Conv2D name=conv2d_4, built=True>, <BatchNormalization name=batch_normalization_4, built=True>, <SeparableConv2D name=block3_sepconv1, built=True>, <BatchNormalization name=block3_sepconv1_bn, built=True>, <SeparableConv2D name=block3_sepconv2, built=True>, <BatchNormalization name=block3_sepconv2_bn

In [11]:
# Load the best frozen model again before fine-tuning
model = keras.models.load_model("best_frozen_model.keras")

# Unfreeze the last 8 layers and add L2 regularization
for layer in model.layers[-8:]:
    layer.trainable = True
    if hasattr(layer, "kernel_regularizer"):
        layer.kernel_regularizer = regularizers.l2(5e-4)  # Apply L2 regularization

# Modify last Dense layer to include Dropout
x = model.layers[-2].output  # Get last Dense layer
dropout = Dropout(0.3)(x)  # Apply dropout before final classification
output = model.layers[-1](dropout)  # Keep the original output layer


# Set early stopping
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5, restore_best_weights=True)

# Rebuild model with dropout
model = keras.Model(inputs=model.input, outputs=output)

# Recompile with a lower learning rate
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Fine-tune the model
history_fine_tune = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size=32, callbacks=[earlystopping])

# Save the fine-tuned model
model.save("best_8layers_unfrozen_model.h5")

ValueError: File not found: filepath=best_frozen_model.keras. Please ensure the file is an accessible `.keras` zip file.

In [21]:
print(history_fine_tune.history.keys())  # See tracked metrics
print(history_fine_tune.history['val_loss'])  # Print validation loss for all epochs


dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])
[1.5351085662841797, 1.5267738103866577, 1.514609456062317]
